In [50]:
# Fix the imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [51]:
import cv2
def view_image(image, windowname='image_view'):
    cv2.imshow(windowname, image)
    cv2.waitKey(0)

In [52]:
import cv2
import numpy as np
import imutils
from imutils.perspective import four_point_transform

def find_board(image):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(img_gray, (3,3), 3)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, element)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, element)
    cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # initialize a contour that corresponds to the puzzle outline
    puzzleCnt = None
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if our approximated contour has four points, then we can
        # assume we have found the outline of the puzzle
        if len(approx) == 4:
            puzzleCnt = approx
            break
    puzzle_warped = four_point_transform(img_gray, puzzleCnt.reshape(4, 2))
    puzzle_warped_thresh = four_point_transform(thresh, puzzleCnt.reshape(4, 2))
    return puzzle_warped, puzzle_warped_thresh

In [53]:
def zoom_image(img, zoom):
    y, x = img.shape
    return img[0 + int(y*zoom): y - int(y*zoom),0 + int(x*zoom): x - int(x*zoom)]

In [54]:
def check_cell(cell):
    y, x = cell.shape
    percent_filled = cv2.countNonZero(zoom_image(cell, 0.2)) / float(y * x)
    return percent_filled < 0.08

In [55]:
image = cv2.imread('./data/sudoku_2.jpeg')
image_warped, image_warped_thresh = find_board(image)
view_image(image_warped)

In [56]:
import pytesseract
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
def pred_number(image) -> int:
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_GRAY2RGB))
    return pytesseract.image_to_string(img, config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')

In [68]:
# initialize our 9x9 Sudoku board
board = np.zeros((9, 9), dtype="int")
# a Sudoku puzzle is a 9x9 grid (81 individual cells), so we can
# infer the location of each cell by dividing the warped image
# into a 9x9 grid
stepX = image_warped.shape[1] // 9
stepY = image_warped.shape[0] // 9
# loop over the grid locations
for y in range(0, 9):
    for x in range(0, 9):
        # compute the starting and ending (x, y)-coordinates of the
        # current cell
        startX = x * stepX
        startY = y * stepY
        endX = (x + 1) * stepX
        endY = (y + 1) * stepY

        # extract the digit from the cell
        cell_thresh = image_warped_thresh[startY:endY, startX:endX]
        is_empty = check_cell(cell_thresh)
        # verify that the digit is not empty
        if not is_empty:
            cell = image_warped[startY:endY, startX:endX]
            pred = pred_number(cell)
            # pred = ocr_cell(cell)['predicted_label'][0]
            if pred:
                board[y, x] = pred[0]

In [69]:
board

array([[8, 0, 0, 0, 1, 0, 0, 0, 9],
       [0, 5, 0, 1, 0, 0, 0, 4, 0],
       [0, 0, 4, 0, 9, 0, 0, 0, 0],
       [0, 6, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 8, 0, 6, 0, 1, 0, 7],
       [0, 4, 0, 5, 0, 0, 0, 9, 0],
       [0, 0, 7, 0, 4, 0, 0, 0, 0],
       [0, 8, 0, 1, 0, 9, 0, 4, 0],
       [1, 0, 0, 0, 5, 0, 0, 0, 8]])

In [ ]:
from src.solver.plp_solver import SudokuSolver

sudoku_solver = SudokuSolver()

In [ ]:
sudoku_solver.solve(
    board
)

array([[8, 3, 6, 9, 4, 2, 5, 7, 1],
       [4, 1, 9, 5, 7, 8, 6, 3, 2],
       [5, 7, 2, 6, 3, 1, 4, 9, 8],
       [1, 9, 3, 2, 6, 7, 8, 4, 5],
       [6, 4, 5, 8, 9, 3, 1, 2, 7],
       [7, 2, 8, 4, 1, 5, 9, 6, 3],
       [3, 5, 4, 1, 2, 6, 7, 8, 9],
       [9, 8, 7, 3, 5, 4, 2, 1, 6],
       [2, 6, 1, 7, 8, 9, 3, 5, 4]])